In [1]:
import json
from collections import defaultdict
import pandas as pd
import numpy as np
from tqdm import tqdm
from haversine import haversine

In [2]:
# Load data from the CSV file into a new DataFrame
data = pd.read_csv('../model/gps_data2.csv')

In [4]:
data = data[data['Latitude'] >= 39.75]
data = data[data['Latitude'] <= 40.1]
data = data[data['Longitude'] >= 116.18]
data = data[data['Longitude'] <= 116.6]

In [5]:
def calculate_distance(coord1, coord2):
	return haversine(coord1, coord2)

In [17]:
meeting_points = defaultdict(list)
meeting_distance_threshold = 0.1
meeting_info_list = []

for index, row in tqdm(data.iterrows(), desc="main data"):
	taxi_id = row["Taxi ID"]
	location = (row["Latitude"], row["Longitude"])

	if meeting_points[location]:
		for other_taxi_id in meeting_points[location]:
			if taxi_id != other_taxi_id:
				other_location = (data.loc[data["Taxi ID"] == other_taxi_id, "Latitude"].iloc[0], data.loc[data["Taxi ID"] == other_taxi_id, "Longitude"].iloc[0])

				distance = calculate_distance(location, other_location)
				if distance <= meeting_distance_threshold:
					meeting_info = {
						"Taxi1": int(other_taxi_id),
						"Taxi2": int(taxi_id),
						"Location": {
							"Latitude": location[0],
							"Longitude": location[1]
						},
						"Distance (meters)": distance,
						"Timestamp1": row["Timestamp"],
						"Timestamp2": data.loc[data["Taxi ID"] == other_taxi_id, "Timestamp"].iloc[0]
					}

					meeting_info_list.append(meeting_info)

	meeting_points[location].extend([taxi_id])

main data: 28157it [03:35, 130.89it/s]


KeyboardInterrupt: 

In [7]:
with open('../files/edges/tdrive.json', 'w') as json_file:
    json.dump(meeting_info_list, json_file, indent=4)